In [4]:
import os
from maths_objects import (
    Formulas, SiteHierarchies, DataSource, FrontMatter, FormulaTable, PageTabs, 
    FormulaTableType, FormulaTableTypeSimple, FormulaTableTypeCalculus, FormulaFile)
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)

In [5]:
def get_financial_summary_df(formulas_input_df):
    """Returns a summary of financial mathematic formulas displayed by
    arithmetic vs geometric sequence
    """
    financial_df = (formulas_input_df[
        (formulas_input_df['Category'] == 'Financial mathematics') &
        (formulas_input_df['State'] == 'NSW') &
        (
            (formulas_input_df['Subcategory_1'] == 'Arithmetic sequence') |
            (formulas_input_df['Subcategory_1'] == 'Geometric sequence')
        )
    ])
    if len(financial_df.index) == 0:
        return None

    financial_df = financial_df[['Subcategory_1', 'Subcategory_2', 'Formula']]
    financial_df['temp_aggregator'] = 1
    financial_df = pd.pivot_table(data=financial_df, values='Formula',
                                  columns='Subcategory_1',
                                  index='Subcategory_2', aggfunc=lambda x: x)
    financial_df.index.name = None
    financial_df.columns.name = None

    # Convert index to categorical data to enable custom sort order
    financial_df.index = pd.Categorical(
        financial_df.index,
        ['Recursive definition', 'n-th term', 'Sum of first n terms',
         'Limiting sum'])
    financial_df = financial_df.sort_index()
    return financial_df


display(get_financial_summary_df(formulas_by_year.to_dataframe()))


,Arithmetic sequence,Geometric sequence
Recursive definition,$ T_n = T_{n-1} + d $,$T_n = rT_{n-1}$
n-th term,$T_n = a + (n-1)d$,$T_n = ar^{n-1}$
Sum of first n terms,$S_n = \dfrac{n}{2}[2a + (n-1)d] = \dfrac{n}{2...,$S_n = \dfrac{a(1-r^n)}{1-r} = \dfrac{a(r^n-1)...
Limiting sum,NaN,"$S=\dfrac{a}{1-r},\ \ |r|<1$"
